# Notebook 14: SOTA Perception & Advanced Metrics (Patent-Ready)

This notebook demonstrates the **Advanced Perception Module** (Phase 9) and **Novel Safety Metrics**.

**Algorithms**:
1. **Euclidean Clustering (KD-Tree)**: Fast segmentation of 3D Lidar data.
2. **PCA Bounding Boxes**: Robust orientation estimation using covariance eigenvectors.
3. **Action Jitter (AJ)**: Quantifying control smoothness.
4. **Time-to-Preempt (TTP)**: Physics-based safety margin.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# Add src to path
sys.path.append(os.path.abspath('../src'))

from perception.point_cloud_processor import PointCloudProcessor
from metrics.advanced_safety import AdvancedSafetyMetrics

## 1. 3D Perception Demo

In [ ]:
# Generate Synthetic Point Cloud (Hospital Room)
def generate_box(center, size, n_points=200):
    # Random points inside a box
    pts = np.random.uniform(-size/2, size/2, (n_points, 3))
    return pts + center

# 1. Bed (Rectangular)
bed_pts = generate_box(center=np.array([2, 2, 0.5]), size=np.array([2, 1, 1]))
# 2. Person (Tall Cylinder-ish)
person_pts = generate_box(center=np.array([4, 3, 0.9]), size=np.array([0.5, 0.5, 1.8]))
# 3. Wall
wall_pts = generate_box(center=np.array([0, 5, 1.5]), size=np.array([10, 0.2, 3]))

cloud = np.vstack([bed_pts, person_pts, wall_pts])

processor = PointCloudProcessor(cloud)

# Clustering (DBSCAN)
clusters = processor.cluster_dbscan(eps=0.8, min_points=10)
print(f"Found {len(clusters)} clusters.")

## 2. OBB Visualization (PCA)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

colors = ['r', 'g', 'b', 'y']

for i, cluster in enumerate(clusters):
    # Plot Points
    ax.scatter(cluster.points[:,0], cluster.points[:,1], cluster.points[:,2], s=5, c=colors[i%len(colors)], alpha=0.5)
    
    # Compute & Plot OBB
    obb = cluster.compute_obb()
    
    # Draw OBB Wireframe
    # (Simplified: just plotting center and main axis for demo)
    ax.scatter(obb.center[0], obb.center[1], obb.center[2], c='k', marker='x', s=100)
    
    # Main Axis vector
    vec = obb.rotation[:, 0] * obb.extent[0]
    ax.quiver(obb.center[0], obb.center[1], obb.center[2], vec[0], vec[1], vec[2], color='k', length=1.0)

ax.set_title("SOTA Perception: Euclidean Clustering + PCA OBB")
plt.show()

## 3. Action Jitter Analysis
Analyze smoothness of a sample trajectory.

In [ ]:
# Load sample log from Dataset
log_df = pd.read_csv('../data/dataset_v0.1/world_test/episodes/episode_00_log.csv')

metrics_engine = AdvancedSafetyMetrics()
res = metrics_engine.compute_metrics(log_df)

print("=== Advanced Safety Metrics ===")
print(f"Action Jitter Score: {res.aj_score:.4f} (Lower = Smoother)")
print(f"Min Time-to-Preempt: {res.min_ttp} s")